In [1]:
import pandas as pd
import os
import yaml 
import mlflow
import mlflow.data
import mlflow.sklearn

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
# --- CAMBIO CLAVE 1: Importar SVC (Support Vector Classifier) ---
from sklearn.svm import SVC 

# --- IMPORTACIONES DE MÉTRICAS (sin cambios) ---
from sklearn.metrics import (
    accuracy_score,
    classification_report,
    confusion_matrix,
    precision_score,
    recall_score,
    f1_score
)

# --- CONFIGURACIÓN DE RUTAS (sin cambios) ---
DATASET_PATH = '../data/raw/student_entry_performance_original.csv' 
DATASET_NAME = 'student_entry_clean'

# --- 1. CARGA DE DATOS (Mismos pasos) ---
try:
    df = pd.read_csv(DATASET_PATH)
    print(f"Dataset cargado desde: {DATASET_PATH}")
except FileNotFoundError:
    print(f"ERROR: Archivo no encontrado en {DATASET_PATH}. Revisa la ruta.")
    exit()

# --- 2. LÓGICA DVC: OBTENER EL HASH (Mismos pasos) ---
dvc_digest = None
dvc_file_path = DATASET_PATH + ".dvc"
if os.path.exists(dvc_file_path):
    try:
        with open(dvc_file_path, 'r') as f:
            dvc_data = yaml.safe_load(f)
        if 'outs' in dvc_data and dvc_data['outs']:
            dvc_digest = dvc_data['outs'][0].get('md5') 
            if not dvc_digest:
                dvc_digest = dvc_data['outs'][0].get('checksum') 
        print(f"DVC Digest encontrado: {dvc_digest}")
    except Exception as e:
        print(f"ADVERTENCIA: No se pudo leer el archivo DVC. Error: {e}")

# --- 3. PREPARACIÓN DE DATOS Y SPLIT (Mismos pasos) ---
X = df.drop(columns=['Performance']) 
y = df['Performance']
cat_cols = ['Gender','Caste','coaching','time','Class_ten_education','twelve_education','medium','Class_ X_Percentage','Class_XII_Percentage','Father_occupation','Mother_occupation']

le = LabelEncoder()
y_enc = le.fit_transform(y)

X_train, X_test, y_train, y_test = train_test_split(X, y_enc, test_size=0.2, stratify=y_enc, random_state=42)


# ---------------------------------------------------------------------
# --- 4. CONFIGURACIÓN DEL PIPELINE BASE Y LA GRILLA DE BÚSQUEDA ---
# ---------------------------------------------------------------------

# Pipeline base
preprocessor = ColumnTransformer(
    transformers=[('ohe', OneHotEncoder(handle_unknown='ignore'), cat_cols)],
    remainder='drop'
)
# --- CAMBIO 2: Reemplazamos el clasificador por SVC ---
pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor), 
    ('clf', SVC(random_state=888, probability=True)) # 'probability=True' es útil para métricas más avanzadas, pero hace el entrenamiento más lento.
])

# --- CAMBIO 3: Definimos la Grilla de Parámetros para SVC ---
# Nota: SVC con GridSearch puede ser lento. Se usan pocos valores.
param_grid = {
    'clf__C': [0.1, 1, 10],            # Parámetro de regularización
    'clf__kernel': ['rbf', 'poly'],    # Tipo de kernel
    'clf__gamma': ['scale', 'auto']    # Coeficiente de kernel para 'rbf', 'poly', etc.
}

# Configuración del Grid Search (sin cambios)
grid_search = GridSearchCV(
    estimator=pipeline,
    param_grid=param_grid,
    scoring='f1_weighted', 
    cv=5,                 
    verbose=2,
    n_jobs=1
)

# --- 5. ENTRENAMIENTO Y PREDICCIONES (Ejecución del Grid Search) ---
print("\nIniciando Grid Search...")
grid_search.fit(X_train, y_train)
print("Grid Search finalizado.")


# ---------------------------------------------------------------------
# --- 6. REGISTRO DE TODAS LAS CORRIDAS EN MLFLOW ---
# ---------------------------------------------------------------------

mlflow.set_tracking_uri("http://127.0.0.1:5001")
# --- CAMBIO 4: Nuevo nombre de experimento para SVC ---
EXPERIMENT_NAME = "svc_GridSearch" 
mlflow.set_experiment(EXPERIMENT_NAME)


# Iterar sobre CADA resultado de CV (Cross-Validation)
for i, (mean_score, std_score, params) in enumerate(zip(
    grid_search.cv_results_['mean_test_score'],
    grid_search.cv_results_['std_test_score'],
    grid_search.cv_results_['params']
)):
    
    with mlflow.start_run(run_name=f"run_{i+1}_SVC_GridSearch", nested=True) as run:
        print(f"Registrando corrida {i+1} con parámetros: {params}")

        # --- CORRECCIÓN: LIMPIEZA DE PARÁMETROS ---
        clf_params = {k.replace('clf__', ''): v for k, v in params.items()}
        # ------------------------------------------
        
        # --- CAMBIO 5: Reconstruir el pipeline con SVC ---
        current_pipeline = Pipeline(steps=[
            ('preprocessor', preprocessor), 
            ('clf', SVC(random_state=888, probability=True, **clf_params)) 
        ])
        
        # Entrenar el modelo con el conjunto de entrenamiento COMPLETO
        current_pipeline.fit(X_train, y_train)
        
        # --- Predicciones y Cálculo de Métricas (en el conjunto de prueba) ---
        y_pred_test = current_pipeline.predict(X_test)
        
        # Calcular Métricas (código sin cambios)
        acc_test = accuracy_score(y_test, y_pred_test)
        f1_micro = f1_score(y_test, y_pred_test, average='micro')
        f1_macro = f1_score(y_test, y_pred_test, average='macro')
        f1_weighted = f1_score(y_test, y_pred_test, average='weighted')
        report_text = classification_report(y_test, y_pred_test, target_names=le.classes_)
        
        metrics = {
            "cv_f1_weighted_mean": mean_score, 
            "cv_f1_weighted_std": std_score,
            "test_acc": acc_test,
            "test_f1_weighted": f1_weighted,
            "test_f1_macro": f1_macro,
        }
        
        # --- Registro en MLflow (sin cambios, solo se actualizan los datos) ---
        mlflow.log_params(params)
        mlflow.log_metrics(metrics)
        
        # Guardar modelo (artefacto)
        # --- CAMBIO 6: Actualizar el nombre del artefacto ---
        mlflow.sklearn.log_model(current_pipeline, "svc_pipeline") 
        
        # Registrar Dataset (sin cambios)
        mlflow_dataset = mlflow.data.from_pandas(
            df=df, source=DATASET_PATH, targets=y.name, name=DATASET_NAME, digest=dvc_digest)
        mlflow.log_input(mlflow_dataset, context="training") 

        # Registrar Classification Report (sin cambios)
        temp_report_path = f"classification_report_run_{i+1}.txt"
        with open(temp_report_path, "w") as f:
            f.write(report_text)
        mlflow.log_artifact(temp_report_path, artifact_path="report")
        os.remove(temp_report_path)

        # Identificación del Mejor Modelo (sin cambios)
        if mean_score == grid_search.best_score_:
              print(f"!!! Este es el mejor modelo (F1-Weighted CV: {mean_score:.4f}) !!!")
              mlflow.set_tag("best_run", "True")


print("\n--- RESUMEN FINAL DE GRID SEARCH ---")
print(f"El mejor F1-Weighted (CV) es: {grid_search.best_score_:.4f}")
print(f"Los mejores parámetros son: {grid_search.best_params_}")

Dataset cargado desde: ../data/raw/student_entry_performance_original.csv
DVC Digest encontrado: fe5c6bdf2fe1d5c36afc295a345fccab

Iniciando Grid Search...
Fitting 5 folds for each of 12 candidates, totalling 60 fits
[CV] END ......clf__C=0.1, clf__gamma=scale, clf__kernel=rbf; total time=   0.0s
[CV] END ......clf__C=0.1, clf__gamma=scale, clf__kernel=rbf; total time=   0.0s
[CV] END ......clf__C=0.1, clf__gamma=scale, clf__kernel=rbf; total time=   0.0s
[CV] END ......clf__C=0.1, clf__gamma=scale, clf__kernel=rbf; total time=   0.0s
[CV] END ......clf__C=0.1, clf__gamma=scale, clf__kernel=rbf; total time=   0.0s
[CV] END .....clf__C=0.1, clf__gamma=scale, clf__kernel=poly; total time=   0.0s
[CV] END .....clf__C=0.1, clf__gamma=scale, clf__kernel=poly; total time=   0.0s
[CV] END .....clf__C=0.1, clf__gamma=scale, clf__kernel=poly; total time=   0.0s
[CV] END .....clf__C=0.1, clf__gamma=scale, clf__kernel=poly; total time=   0.0s
[CV] END .....clf__C=0.1, clf__gamma=scale, clf__kerne

2025/11/02 20:09:45 INFO mlflow.tracking.fluent: Experiment with name 'svc_GridSearch' does not exist. Creating a new experiment.


Grid Search finalizado.
Registrando corrida 1 con parámetros: {'clf__C': 0.1, 'clf__gamma': 'scale', 'clf__kernel': 'rbf'}


c:\Users\Anuar\Documents\Maestria Inteligencia artificial\MLOPs Prueba\Cookie cutter 2\proyecto_mlops_equipo_56\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\Anuar\Documents\Maestria Inteligencia artificial\MLOPs Prueba\Cookie cutter 2\proyecto_mlops_equipo_56\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\Anuar\Documents\Maestria Inteligencia artificial\MLOPs Prueba\Cookie cutter 2\proyecto_mlops_equipo_56\.venv\Lib\site-packages\sklearn\metrics\_classification.py:153

Registrando corrida 2 con parámetros: {'clf__C': 0.1, 'clf__gamma': 'scale', 'clf__kernel': 'poly'}


...

Registrando corrida 11 con parámetros: {'clf__C': 10, 'clf__gamma': 'auto', 'clf__kernel': 'rbf'}


2025/11/02 20:12:37 WARNING mlflow.models.model: Input example should be provided to infer model signature if the model signature is not provided when logging the model.
c:\Users\Anuar\Documents\Maestria Inteligencia artificial\MLOPs Prueba\Cookie cutter 2\proyecto_mlops_equipo_56\.venv\Lib\site-packages\mlflow\data\dataset_source_registry.py:150: UserWarning: Failed to determine whether UCVolumeDatasetSource can resolve source information for '../data/raw/student_entry_performance_original.csv'. Exception: 
  return _dataset_source_registry.resolve(
c:\Users\Anuar\Documents\Maestria Inteligencia artificial\MLOPs Prueba\Cookie cutter 2\proyecto_mlops_equipo_56\.venv\Lib\site-packages\mlflow\data\dataset_source_registry.py:150: UserWarning: The specified dataset source can be interpreted in multiple ways: LocalArtifactDatasetSource, LocalArtifactDatasetSource. MLflow will assume that this is a LocalArtifactDatasetSource source.
  return _dataset_source_registry.resolve(
2025/11/02 20:12

Registrando corrida 12 con parámetros: {'clf__C': 10, 'clf__gamma': 'auto', 'clf__kernel': 'poly'}


2025/11/02 20:12:54 WARNING mlflow.models.model: Input example should be provided to infer model signature if the model signature is not provided when logging the model.
c:\Users\Anuar\Documents\Maestria Inteligencia artificial\MLOPs Prueba\Cookie cutter 2\proyecto_mlops_equipo_56\.venv\Lib\site-packages\mlflow\data\dataset_source_registry.py:150: UserWarning: Failed to determine whether UCVolumeDatasetSource can resolve source information for '../data/raw/student_entry_performance_original.csv'. Exception: 
  return _dataset_source_registry.resolve(
c:\Users\Anuar\Documents\Maestria Inteligencia artificial\MLOPs Prueba\Cookie cutter 2\proyecto_mlops_equipo_56\.venv\Lib\site-packages\mlflow\data\dataset_source_registry.py:150: UserWarning: The specified dataset source can be interpreted in multiple ways: LocalArtifactDatasetSource, LocalArtifactDatasetSource. MLflow will assume that this is a LocalArtifactDatasetSource source.
  return _dataset_source_registry.resolve(
2025/11/02 20:13


--- RESUMEN FINAL DE GRID SEARCH ---
El mejor F1-Weighted (CV) es: 0.4786
Los mejores parámetros son: {'clf__C': 1, 'clf__gamma': 'scale', 'clf__kernel': 'rbf'}
